In [ ]:
import sys
sys.path.append(r'D:\Workspace\Python\Project\NeuralNetwork')

In [19]:
class Model:
    def __init__(self):
        pass
    def forward(self):
        raise NotImplementedError()
    def backward(self):
        self.all_layer()
    def all_layer(self):
        layers = []
        for attr in self.__dict__.values():
            print(attr)

In [21]:
from layers import Dense
class MyModel(Model):
    def __init__(self):
        super().__init__()
        self.layer1 = [1,2,3]
        self.layer2 = 5
        self.layer3 = Dense(10,'relu')
    def forward(self):
        print('a')
        return 
    

In [22]:
a = MyModel()
a.backward()